# Supervised Learning II
by Mauricio Araya
## (A.K.A. Supervised Learning Lab)

<b>Credit:</b> Guillermo Cabrera, Matthew Graham, and Scikit Learn

(Yes... I will force you to code now...)

## 1.- Regularized Regression... or ridding alone

### Objective
* Understand the effect of the regularization parameters
* Compare Ridge and Lasso
* Warm up!

### Regularization Reminder (slide)

## Exercise
### a) Forging your own Galaxy Photometry/Redshift dataset
* Use my Tuesday's notebook to load the `sdss_gal.csv`
* Downsample the data (use $n=10000$ for example)
* Divide into training and test/validation
* Select the 'u-g' feature (and hack it to be a matrix)

In [1]:
import pandas as pd
import numpy as np
galaxy_feat = pd.read_csv('sdss_gal.csv', low_memory=False)


### b) Use the scikit-learn to perform a ridge regression 
* Use now a polynomial model (degree = 10 for example)
* Plot your data and the curve line (use the `.predict()` function, not manually)
* Plot the parameters of the regression using a bar plot 

In [2]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.preprocessing import PolynomialFeatures


### c) See how regularization works
* Pack the code into a function that recieves the degree of the polynomial and the alpha parameter for regularizing Ridge.
* This function should overplot the regression line to the data, and in a separate plot the parameters of the regression using a bar plot.
* Use interact to play with the two values ($d \in [2,15]$, $\alpha \in [0,1]$)
* Do the same for LASSO regularization

In [3]:
def train_and_plot(d=10,a=0.0):
    pass

## 2.- Linear Regression... without training wheels

### Objective

As an exercise lets do a linear regression without training wheels:
* <b>Left wheel</b>: $x \in \mathbb{R}$ --> $\mathbf{x} \in \mathbb{R}^n$ variable
* <b>Right wheel</b>: `scikit-learn` package --> just `pandas`, `numpy` and `scipy`

### Theory
What is a linear model?
$$ y = f(\mathbf{x};\mathbf{w}) = \sum_j w_j \phi_j(\mathbf{x}) = \mathbf{w}^\textrm{T} \boldsymbol{\phi}(\mathbf{x}) $$

Examples:
* Polinomial: $\phi_j(\mathbf{x}) = \|\mathbf{x}\|^j$
* Gaussian: $\phi_j(\mathbf{x}) = \exp\left\{\frac{- \|\mathbf{x} - \boldsymbol{\mu}_j\|^2}{2s^2}\right\}$
* Sigmoidal: $\phi_j(\mathbf{x}) = \sigma\left(\frac{\|\mathbf{x} - \boldsymbol{\mu}_j\|}{s}\right)= \frac{1}{1 + \exp\left( \frac{-\|\mathbf{x} - \boldsymbol{\mu}_j\|}{s}\right)}$

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

def show_gaussian(mu=0.0,s=1.0):
    a = np.arange(-10, 10, 0.1)
    plt.plot(a, np.exp(-(a-mu)**2/(2*s**2)))
interact(show_gaussian,mu=(-2.0,2.0),s=(0.1,10))

interactive(children=(FloatSlider(value=0.0, description='mu', max=2.0, min=-2.0), FloatSlider(value=1.0, description='s', max=10.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.show_gaussian>

In [5]:
def show_sigmoid(mu=0.0,s=1.0):
    a = np.arange(-10, 10, 0.1)
    plt.plot(a, 1./(1. + np.exp(-(a-mu)/s)))
interact(show_sigmoid,mu=(-2.0,2.0),s=(0.1,10))

interactive(children=(FloatSlider(value=0.0, description='mu', max=2.0, min=-2.0), FloatSlider(value=1.0, description='s', max=10.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.show_sigmoid>

Imagine you fix every $\mu_j$ and $s$. 

<b>How we can learn</b> $\mathbf{w}$ (assuming Gaussian noise $\epsilon \sim N(\mu,\sigma)$)?

$$\begin{align} Pr(Y \mid \mathbf{X},\mathbf{y}) &= \prod_i \mathcal{N}(Y = y_i \mid f(\mathbf{x}_i;\mathbf{w}),\sigma)\\ &= \prod_i \mathcal{N}(Y = y_i \mid \mathbf{w}^\textrm{T} \boldsymbol{\phi}(\mathbf{x}_i),\sigma)\end{align} $$

We want to maximize this probability

$$ \begin{align} \ln Pr(Y \mid \mathbf{X},\mathbf{y}) & = \sum_i \ln \mathcal{N}(Y = y_i \mid \mathbf{w}^\textrm{T} \boldsymbol{\phi}(\mathbf{x}_i),\sigma)\\ & \propto \sum_i\left( y_i - \mathbf{w}^\textrm{T} \boldsymbol{\phi}(\mathbf{x}_i)\right)^2 \end{align} $$

If we compute the gradient of this
$$ \begin{align} \nabla\ln Pr(Y \mid \mathbf{X},\mathbf{y}) & = \sum_i\left( y_i - \mathbf{w}^\textrm{T}\boldsymbol{\phi}(\mathbf{x}_i)\right)\boldsymbol{\phi}(\mathbf{x}_i)^\textrm{T} \\
\nabla\ln Pr(Y \mid \mathbf{X},\mathbf{y}) &= \mathbf{0} \\
\sum_i y_i \boldsymbol{\phi}(\mathbf{x}_i)^\textrm{T} &= \mathbf{w}^\textrm{T}\sum_i\boldsymbol{\phi}(\mathbf{x}_i)\boldsymbol{\phi}(\mathbf{x}_i)^\textrm{T}\\
\mathbf{y}^\textrm{T}\boldsymbol{\Phi} &= \mathbf{w}^\textrm{T}\boldsymbol{\Phi}\boldsymbol{\Phi}^\textrm{T}\\
\mathbf{w} &= (\boldsymbol{\Phi}^\textrm{T}\boldsymbol{\Phi})^{-1}\boldsymbol{\Phi}^\textrm{T}\mathbf{y}
\end{align}$$

The $\boldsymbol{\Phi}$ is called the <b>Design Matrix</b> and have the form:
$$\begin{bmatrix}
1 & \phi_1(\mathbf{x}_1) & \phi_2(\mathbf{x}_1) & \ldots & \phi_m(\mathbf{x}_1)\\
1 & \phi_1(\mathbf{x}_2) & \phi_2(\mathbf{x}_2) & \ldots & \phi_m(\mathbf{x}_2)\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
1 & \phi_1(\mathbf{x}_n) & \phi_2(\mathbf{x}_n) & \ldots & \phi_m(\mathbf{x}_n)
\end{bmatrix}$$

## Exercise
### a) Use now ALL the dimensions now and select $\mu$'s 
* Check if you have all the dimensions of the training and validation sets (not one like in the previous example)
* Resample to $n=1000$ o similar for working
* Select $m=20$ points of the training sample to use a $\mu$ values
* It will be handy for later doing this in a function, recieving the number of total samples $(n,m)$

In [6]:
def get_matrices(n,m):
    pass

### b) Construct your own design matrix
* Choose a not so trivial kernel (i.e. Gaussian/Sigmoidal)
* Do not obsess with speed... yet. Just give a solution.
* If you have dissmissed the previous recommendation, use the L2-norm $\|\cdot\|_2$ and remember that
$$ (\mathbf{a} - \mathbf{b})^\textrm{T}(\mathbf{a} - \mathbf{b}) = \mathbf{a}^\textrm{T}\mathbf{a} + \mathbf{b}^\textrm{T}\mathbf{b} - 2\mathbf{a}^\textrm{T}\mathbf{b} $$ and remember to round up to 10 decimals
* (Optional) Plot the 20 first rows of the matrix using `plt.imshow`

### c) Train and Predict
* Get a design matrix
* Compute $\mathbf{w} = (\boldsymbol{\Phi}^\textrm{T}\boldsymbol{\Phi})^{-1}\boldsymbol{\Phi}^\textrm{T}\mathbf{y}$ using `scipy.linal` package for computing the inverse. Remember that this model is not regularized, so compute the pseudo-inverse.
* Get the "predictions" for the training data $\mathbf{w}^\textrm{T}\boldsymbol{\Phi}_\mathbf{X}$
* Get the predictions for the test/validation data $\mathbf{w}^\textrm{T}\boldsymbol{\Phi}_\mathbf{X'}$
* Do a scatter plot of the real redshifts of the training set against the 'u-g' dimension in blue
* Use the same plot to show the predicted redshifts values of the training set.
* Create a new figure for the test/validation data
* Did you put all this in a function?

### d) Computing the RMS errors
* Compute the RMS error for the training and validation/test set: $RMS = \sqrt{\frac{\sum_i^n (y_i - y_i')^2}{n}}$
* Plot the error for $s \in [0.1,5]$
* Plot the error for $n \in [100,10000]$ step = 100
* Plot the error for $m \in [10,100] $ step = 1

## 3.- Logistic "Regression"... into the bikeway
### Objective
* Use regression models to classify (everything is connected)
* Exercise the use of metrics
* Learn the importance of understanding the model

### Theory

Let us explore the following idea:
$$\sigma(\mathbf{w}^\textrm{T}\mathbf{x}) = \frac{1}{1+\exp(-\mathbf{w}^\textrm{T}\mathbf{x})}$$

In [7]:
def logistic2D(w1=1,w2=1,w3=1):
    w = [w1, w2, w3]
    x1_g, x2_g = np.meshgrid(np.arange(-5., 5.0, 0.1),np.arange(-5., 5.0, 0.1))
    y = w[0] + w[1]*x1_g + w[2]*x2_g
    plt.contourf(x1_g, x2_g, 1./(1.+np.exp(-y)), cmap=plt.cm.seismic, levels = np.arange(0, 1.1, 0.05))
    plt.xlim([-5,5])
    plt.ylim([-5,5])
    plt.colorbar()
interact(logistic2D,w1=(-5,5),w2=(0.001,1.),w3=(0.001,1.))

interactive(children=(IntSlider(value=1, description='w1', max=5, min=-5), FloatSlider(value=1.0, description='w2', max=1.0, min=0.001), FloatSlider(value=1.0, description='w3', max=1.0, min=0.001), Output()), _dom_classes=('widget-interact',))

<function __main__.logistic2D>

Lets use our linear model under a sigmoid function $\sigma$ to separate classes then! 

Let $\phi(\boldsymbol{x}) = \boldsymbol{\phi}$

$Pr(Y = c_1 \mid \boldsymbol{x}) = \sigma(\boldsymbol{w}^\top\boldsymbol{\phi})$

Reciprocaly

$Pr(Y = c_2 \mid \boldsymbol{x}) = 1-\sigma(\boldsymbol{w}^\top\boldsymbol{\phi})$

Note that:

$\sigma(-a) = 1 - \sigma(a)$

So...

$\boldsymbol{w}^\top\boldsymbol{\phi} = \ln\left(\frac{Pr(Y=c_1\mid\boldsymbol{x})}{Pr(y=c_2\mid\boldsymbol{x})}\right)$,

Let us define $c_1 = 1$ and $c_2 = 0$ just because... then,

$Pr(Y=y\mid\boldsymbol{x}) = \sigma(\boldsymbol{w}^\top\boldsymbol{\phi})^y (1 - \sigma(\boldsymbol{w}^\top\boldsymbol{\phi})^{1-y}$

If we try to compute the log likelihood for a training data

$$ E(\boldsymbol{w}) = \ln Pr(\mathbf{Y}=\mathbf{y}\mid\boldsymbol{X},\boldsymbol{w}) = \sum_i \left\{y_i \ln( \sigma(\boldsymbol{w}^\top\boldsymbol{\phi}_i)) + (1-y_i)\ln(1 - \sigma(\boldsymbol{w}^\top\boldsymbol{\phi}_i))\right\}$$

Then, the gradient looks nice...

$\nabla E(\boldsymbol{w}) = \sum_{i}(\sigma(\boldsymbol{w}^\top\boldsymbol{\phi}_i)) - y_n)\boldsymbol{\phi} = 0$ 

but is no closed form to solve this, so we use iterative methods to solve them (numerical optimization methods such as Newton-Raphson). 

Obviously, Mr. Scikit Learn have coded that for us... [link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

## Exercise

### a) Load and balance the SDSS stars dataset (RRLyrae)
* Load the dataset and put names to the columns
* Find out how imbalanced is the dataset
* Subsample the class of non-variable stars to the same size of the other class

### b) Train and Evaluate a Logistic Regression Classifier (LRC)
* Train the LRC with the balanced data
* Predict the classes for the same data
* Compute the completeness and contamination 

In [8]:
from sklearn import linear_model

### c) Incrementally include the rest of the data
* Create a function that trains and predicts depending on the number of samples used for subsampling the non-variables class. This function should return the completeness and contamination.
* Plot the behaviour of the completeness and contamination as you increase the number of samples of the non-variables class. I recommend steps of 500. 
* Plot also the parameters of the model using bars
* What is your explanation for this result? can we do something about it?

In [9]:
def get_measures(ns):
    pass